* = 'to do' item for later
* Convert to a python script (for automation)

Step 0: Relevant packages installed (ref 00_requirements.txt) <br>
Step 1: Data downloaded <br>
Step 1a: Nodes 

I. [Load Packages](#Load) clicking on phrase will take you directly to section <br>
II. [Query for Biomedical Node Types in Wikidata](#Query)

## Load 
Packages and modules with relevant functions

In [14]:
from pathlib import Path
from tqdm.autonotebook import tqdm 

from data_tools.df_processing import char_combine_iter 
from data_tools.wiki import node_query_pipeline

Make an empty list for nodes (this will become a populated .csv)

In [15]:
nodes = []

## Query
Biomedically relevant node types in Wikidata (ordered alphabetically)
* Node categories to add or adjust?
* Identifiers within node categories to add or adjust, if relevant (otherwise remove)?
* Make into a for loop (DRY)?

### Anatomy
### Biological Process
### Cellular Component
### Compounds
### Disease
### Genes
### Pathway
### Phenotype
### Protein
### Molecular Function

update and rename mike's data_tools package? others that use it...?
note to change pd to pandas